#### Score : ```0.88```

# Read All Dataset CSV

In [53]:
import os
import pandas as pd
import numpy as np

In [54]:
def remove_all_predict():
    for folder_name in os.listdir("../../Competition_data"):
        if os.path.exists(f"../../Competition_data/{folder_name}/y_predict.csv") is False: continue
        os.remove(f"../../Competition_data/{folder_name}/y_predict.csv")
# use this function to remove all the y_predict.csv
remove_all_predict()

In [55]:
dataset_names=[]
X_trains=[]
y_trains=[]
X_tests=[]
for folder_name in os.listdir("../../Competition_data"):
    dataset_names.append(folder_name)
    X_trains.append(pd.read_csv(f"../../Competition_data/{folder_name}/X_train.csv",header=0))
    y_trains.append(pd.read_csv(f"../../Competition_data/{folder_name}/y_train.csv",header=0))
    X_tests.append(pd.read_csv(f"../../ompetition_data/{folder_name}/X_test.csv",header=0))

## Data Preprocessing & Feature Engineering

In [56]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [57]:
## your code here

def get_number_of_datatype(X_data):
    # Split the features in X_data into numeric_data and categorical_data.
    numeric, categoric = [], []
    for feature in X_data:
        val = float(X_data.loc[i, feature])
        if isinstance(val, float) and val.is_integer():
                categoric.append(feature)
        else:   numeric.append(feature)
    return numeric, categoric

dataset_datatype_cnt = []
for i, _ in enumerate(dataset_names):
    numeric_features, categoric_features = get_number_of_datatype(X_data = X_trains[i])
    dataset_datatype_cnt.append((numeric_features, categoric_features))


## train test split & build Model
You can select an appropriate model and perform corresponding hyperparameter tuning.

In [58]:
from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [59]:
TEST_MODEL = False

In [60]:

# Preprocessing for numerical data
numerical_transformer = StandardScaler()

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

models=[]
for i in range(len(dataset_names)):
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numerical_transformer, dataset_datatype_cnt[i][0]),
            ("cat", categorical_transformer, dataset_datatype_cnt[i][1]),
        ]
    )

    # Define the model
    model = RandomForestClassifier(random_state = 42)
    pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", model)])
    if TEST_MODEL:
        tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(X_trains[i], y_trains[i], test_size=0.2, random_state=42)
        tmp_y_train = tmp_y_train.squeeze()
        tmp_y_test = tmp_y_test.squeeze()
        # Combine preprocessing steps
        
        pipeline.fit(tmp_X_train, tmp_y_train)
        tmp_y_prob = pipeline.predict_proba(tmp_X_test)[:, 1]
        auc = roc_auc_score(tmp_y_test, tmp_y_prob)
    else:
        y_trains[i] = y_trains[i].squeeze()
        pipeline.fit(X_trains[i], y_trains[i])
    models.append(pipeline)



## Inference Model

In [61]:
y_predicts=[]
for i in range(len(dataset_names)):
    y_predict_proba=models[i].predict_proba(X_tests[i])[:, 1]
    df = pd.DataFrame(y_predict_proba, columns=['y_predict_proba'])
    y_predicts.append(df)
    

## Save result

In [62]:
for idx,dataset_name in enumerate(dataset_names):
    df = y_predicts[idx]
    df.to_csv(f'../../Competition_data/{dataset_name}/y_predict.csv', index = False,header = True)